In [ ]:
# If running on Colab for the first time, run this cell
!pip install pandas datetime fpdf2

In [1]:
import pandas as pd
import datetime as dt
from fpdf import FPDF

In [74]:
ieos_abstracts = (
    pd.read_csv("IEOS2023_presentations.csv", encoding="utf_8")
    .fillna("", inplace=False)
    .replace("\u200b", "", regex=True)
    .replace("\xad", "", regex=True)
    .replace("ã\x80\x96", "", regex=True)
    .replace("ã\x80\x97", "", regex=True)
    .replace("â\x80\x94", " - ", regex=True)
    .replace("\u200b", "", regex=True)
    .replace("â\x80\x9c", '"', regex=True)
    .replace("â\x80\x9d", '"', regex=True)
    .replace("â\x88\x92", "-", regex=True)
    .replace("â\x80\x93", "-", regex=True)
    .replace("â\x80\x98", "'", regex=True)
    .replace("â\x80\x99", "'", regex=True)
    .replace("â\x80\x8b", "", regex=True)
    .sort_values(by=["Session_sort", "Presentation_sort"])
)

,Session_sort,Presentation_sort,Session,Room,Time,Authors,Title,Affiliation,Type,Abstract
0,1,1,1 (A),Barrington Room,0915-0930,"A. Magee, G. Misra, P. Lewis, L. Osullivan, S....",Multi-functional EO capabilities of Digital Te...,NCG,Oral,In the recent decades we have been witnessing ...
1,1,2,1 (A),Barrington Room,0930-0945,"J. Hanafin, A. McKinstry, F. Cawkwell, G. Serb...",Earth Observation DataCube for Ireland,ICHEC,Oral,The aim of this project is to build a DataCube...
2,1,3,1 (A),Barrington Room,0945-1000,"V. Hussey, R. McDonnell, E. Bridge, P. Hennell...","Earth Observation Data Collection, Use and pub...",The Office of Public Works (OPW),Oral,A review of the collection and use of Earth Ob...
3,1,4,1 (A),Barrington Room,1000-1015,N. Bolger,ESA Earth Observation research and commerciali...,Enterprise Ireland / Irish Delegation to Europ...,Oral,Presentation of ESA Earth Observation Future E...
4,1,5,1 (A),Barrington Room,1015-1030,"T. McCarthy, D. McInerney, J. Connolly, S. Gre...",Irish National LiDAR Mapping Campaign,Maynooth University,Oral,Light Detection and Ranging (LiDAR) sensor tec...
5,2,1,2 (B),Barrington Room,1100-1130,D. Rocchini,Knocking information theory out of the park: n...,Uni Bologna,KEYNOTE,Ecosystem heterogeneity has been widely recogn...
6,2,2,2 (B),Barrington Room,1130-1145,R. O'Loughlin,The Area Monitoring System - Use of Earth Obse...,Compass Informatics,Oral,This presentation provides an overview of the ...
7,2,3,2 (B),Barrington Room,1145-1200,"G. Smith, S. Kleeschulte, T. Soukup, A. Tellet...",From Prototype to Operational Service: Expansi...,Specto Natura,Oral,Grasslands are important for the maintenance o...
8,2,4,2 (B),Barrington Room,1200-1215,E. Baykiev,Detection of hedgerows using CAMEO platform,Compass Informatics,Oral,The goal of the CAMEO (Creating an Architectur...
9,2,5,2 (B),Barrington Room,1215-1230,C. White,Monitoring Biodiversity in Agricultural Supply...,Trinity College Dublin,Oral,Agrifood corporates are under increasing press...


In [81]:
class PDF(FPDF):
    def __init__(self, orientation, unit, format):
        super().__init__(orientation, unit, format)

    def header(self):
        self.image(self.correct_image, x=0, y=0, w=148)
        self.ln(42)

    def footer(self):
        # Position at 1.5 cm from bottom
        self.set_y(195)
        # set italic font
        self.set_font("DejaVu", "I", 8)
        # Text color in blue
        self.set_text_color(33, 66, 142)
        # Page number
        self.cell(0, 10, "Page " + str(self.page_no()), 0, 0, "C")
        # reset text color to black again
        self.set_text_color(00, 00, 00)

    def what(self, type_of, session, room, pres_time):
        self.set_font("DejaVu", "BI", 8)
        # Text color in blue
        self.set_text_color(33, 66, 142)
        # Title
        self.multi_cell(0, 5, f"{type_of} :: Session {session}, {room} @ {pres_time}")
        # reset text color to black again
        self.set_text_color(00, 00, 00)
        # Line break
        self.ln(2)

    def abs_title(self, ptitle):
        self.set_font("DejaVu", "B", 15)
        # add title
        self.write(h=7, text=ptitle)
        # Line break
        self.ln(8)

    def abs_auth(self, author):
        self.set_font("DejaVu", "I", 8)
        # add author names
        self.multi_cell(0, 5, author)
        # Line break
        self.ln(2)

    def abs_aff(self, aff):
        self.set_font("DejaVuL", "", 6)
        # add affiliation
        self.multi_cell(0, 5, aff)
        # Line break
        self.ln(2)

    def abs_body(self, abstract):
        self.set_font("DejaVu", "", 8)
        # Output justified text
        self.write(h=4, text=abstract)
        # Line break
        self.ln()

    def print_abs(self, title, author, aff, abs, typeof, session, room, pres_time, session_sort):
        if session_sort <= 7:
            self.correct_image = "IEOS2023_HEADER.png"
        else:
            self.correct_image = "IEOS2023_HEADER2.png"
        self.add_page(orientation="P")
        self.what(typeof, session, room, pres_time)
        self.abs_title(title)
        self.abs_auth(author)
        self.abs_aff(aff)
        self.abs_body(abs)

    def add_title_page(self):
        self.correct_image="IEOS2023_p1.png"
        self.add_page(orientation="P")

    def add_contents_page(self, df):
        self.correct_image = "IEOS2023_HEADER.png"
        self.add_page(orientation="P")
        for i in df['Session_sort'].unique():
            if i>=8:
                continue
            session_df = (df[df['Session_sort']==i])
            session = session_df.iloc[0]["Session"]
            room = session_df.iloc[0]["Room"]
            session_df = session_df.drop(
                [
                    "Session_sort",
                    "Presentation_sort",
                    "Session",
                    "Room",
                    "Affiliation",
                    "Type",
                    "Abstract",
                ],
                axis=1,
            )
            # set bold font
            self.set_font("DejaVu", "", 10)
            # Text color in blue
            self.set_text_color(33, 66, 142)
            # Session info
            with pdf.table(borders_layout="NONE", text_align=("LEFT", "LEFT", "LEFT"), col_widths=(20, 40, 60), v_align="TOP", first_row_as_headings=False, line_height=10) as table:
                if session == "Posters":
                    session_df["Time"] = " "
                    data_row = [" ",f"{session}",room]
                else:
                    data_row = [" ",f"Session {session}",room]
                row = table.row()
                for datum in data_row:
                    row.cell(datum)
            # reset text color to black again
            self.set_text_color(00, 00, 00)
            # set bold font
            self.set_font("DejaVu", "", 7)
            with pdf.table(borders_layout="NONE", text_align=("LEFT", "LEFT", "LEFT"), col_widths=(20, 40, 60), v_align="TOP", first_row_as_headings=False, line_height=4) as table:
                for i in session_df.index:
                    data_row = session_df.loc[i, :].values.flatten().tolist() 
                    row = table.row()
                    for datum in data_row:
                        row.cell(datum)            



pdf = PDF(orientation="P", unit="mm", format=(148, 210))
pdf.add_font("DejaVu", "", "DejaVuSans.ttf")
pdf.add_font("DejaVu", "BI", "DejaVuSans-BoldOblique.ttf")
pdf.add_font("DejaVu", "B", "DejaVuSans-Bold.ttf")
pdf.add_font("DejaVuL", "", "DejaVuSans-ExtraLight.ttf")
pdf.add_font("DejaVu", "I", "DejaVuSans-Oblique.ttf")
pdf.set_text_shaping(True)

pdf.add_title_page()

pdf.add_contents_page(ieos_abstracts)

for i in ieos_abstracts.index:
    pdf.print_abs(
        ieos_abstracts.iloc[i]["Title"],
        ieos_abstracts.iloc[i]["Authors"],
        ieos_abstracts.iloc[i]["Affiliation"],
        ieos_abstracts.iloc[i]["Abstract"],
        ieos_abstracts.iloc[i]["Type"],
        ieos_abstracts.iloc[i]["Session"],
        ieos_abstracts.iloc[i]["Room"],
        ieos_abstracts.iloc[i]["Time"],
        ieos_abstracts.iloc[i]["Session_sort"]
    )

pdf.output("ieos_abstracts.pdf")

/tmp/ipykernel_9995/2008367427.py:17: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=0 use new_x=XPos.RIGHT, new_y=YPos.TOP.
  self.cell(0, 10, "Page " + str(self.page_no()), 0, 0, "C")
/tmp/ipykernel_9995/2008367427.py:17: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=0 use new_x=XPos.RIGHT, new_y=YPos.TOP.
  self.cell(0, 10, "Page " + str(self.page_no()), 0, 0, "C")
/tmp/ipykernel_9995/2008367427.py:17: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=0 use new_x=XPos.RIGHT, new_y=YPos.TOP.
  self.cell(0, 10, "Page " + str(self.page_no()), 0, 0, "C")
/tmp/ipykernel_9995/2008367427.py:17: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=0 use new_x=XPos.RIGHT, new_y=YPos.TOP.
  self.cell(0, 10, "Page " + str(self.page_no()), 0, 0, "C")
